In [1]:
pip install datasets pandas numpy matplotlib seaborn


Note: you may need to restart the kernel to use updated packages.


In [2]:
pip install krippendorff


Note: you may need to restart the kernel to use updated packages.


In [3]:
import krippendorff
from datasets import load_dataset
import pandas as pd
import numpy as np


/opt/anaconda3/lib/python3.13/site-packages/pandas/core/computation/expressions.py:22: UserWarning: Pandas requires version '2.10.2' or newer of 'numexpr' (version '2.10.1' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED


In [4]:
dataset = load_dataset("ucberkeley-dlab/measuring-hate-speech")


In [5]:
dataset["train"][0]


{'comment_id': 47777,
 'annotator_id': 10873,
 'platform': 3,
 'sentiment': 0.0,
 'respect': 0.0,
 'insult': 0.0,
 'humiliate': 0.0,
 'status': 2.0,
 'dehumanize': 0.0,
 'violence': 0.0,
 'genocide': 0.0,
 'attack_defend': 0.0,
 'hatespeech': 0.0,
 'hate_speech_score': -3.9,
 'text': 'Yes indeed. She sort of reminds me of the elder lady that played the part in the movie "Titanic" who was telling her story!!! And I wouldn\'t have wanted to cover who I really am!! I would be proud!!!! WE should be proud of our race no matter what it is!!',
 'infitms': 0.81,
 'outfitms': 1.88,
 'annotator_severity': 0.36,
 'std_err': 0.34,
 'annotator_infitms': 1.35,
 'annotator_outfitms': 1.23,
 'hypothesis': -1.1301777576839678,
 'target_race_asian': True,
 'target_race_black': True,
 'target_race_latinx': True,
 'target_race_middle_eastern': True,
 'target_race_native_american': True,
 'target_race_pacific_islander': True,
 'target_race_white': True,
 'target_race_other': False,
 'target_race': True,
 

In [6]:
df = pd.DataFrame(dataset["train"])

In [7]:
religion_cols = [c for c in df.columns if c.startswith("annotator_religion_")]
religion_cols


['annotator_religion_atheist',
 'annotator_religion_buddhist',
 'annotator_religion_christian',
 'annotator_religion_hindu',
 'annotator_religion_jewish',
 'annotator_religion_mormon',
 'annotator_religion_muslim',
 'annotator_religion_nothing',
 'annotator_religion_other']

In [9]:
df[religion_cols].sum().sort_values(ascending=False)


annotator_religion_christian    58573
annotator_religion_nothing      36790
annotator_religion_atheist      27143
annotator_religion_other         8511
annotator_religion_jewish        2171
annotator_religion_buddhist      2138
annotator_religion_mormon        1066
annotator_religion_muslim         969
annotator_religion_hindu          660
dtype: int64

In [32]:
# def score_label(score):
#     if score > 0.5:
#         return "Hate speech"
#     elif score < -1:
#         return "Counter"
#     else:
#         return "Neutral"

# df["score_label"] = df["hate_speech_score"].apply(score_label)




In [18]:
df["hate_speech_score"] = pd.to_numeric(df["hate_speech_score"], errors="coerce")

df["is_hate_speech_score"] = df["hate_speech_score"] > 0.5

religion_cols = [
    'annotator_religion_atheist',
    'annotator_religion_christian',
    'annotator_religion_nothing',
    'annotator_religion_other'
]

rows = []
for rel in religion_cols:
    rel_name = rel.replace("annotator_religion_", "")
    subset = df[df[rel] == 1]

    total_n = len(subset)
    hate_n = subset["is_hate_speech_score"].sum()
    hate_pct = (hate_n / total_n * 100) if total_n > 0 else 0.0

    rows.append({
        "religion": rel_name,
        "n_annotations": total_n,
        "hate_speech_%": round(hate_pct, 2)
    })

hate_speech_by_religion = pd.DataFrame(rows).sort_values("n_annotations", ascending=False)
hate_speech_by_religion


,religion,n_annotations,hate_speech_%
1,christian,58573,36.04
2,nothing,36790,36.11
0,atheist,27143,36.78
3,other,8511,36.40


In [20]:

df["hate_speech_score"] = pd.to_numeric(df["hate_speech_score"], errors="coerce")

df["score_label"] = np.select(
    [
        df["hate_speech_score"] > 0.5,
        df["hate_speech_score"] < -1
    ],
    [
        "Hate Speech",
        "Counter "
    ],
    default="Neutral "
)

df["score_label"].value_counts(dropna=False)


score_label
Counter        53651
Hate Speech    49048
Neutral        32857
Name: count, dtype: int64

In [28]:
religion_cols = [
    'annotator_religion_atheist',
    'annotator_religion_christian',
    'annotator_religion_nothing',
    'annotator_religion_other'
]

label_order = ["Hate Speech", "Neutral ", "Counter "]

rows = []

for rel in religion_cols:
    rel_name = rel.replace("annotator_religion_", "")
    subset = df[df[rel] == 1]

    counts = subset["score_label"].value_counts().reindex(label_order, fill_value=0)
    total = counts.sum()
    perc = (counts / total * 100).round(2) if total > 0 else counts

    rows.append({
        "religion": rel_name,
        "total_annotations": int(total),

        "hate": int(counts["Hate Speech"]),
        "h%": float(perc["Hate Speech"]),

        "neutral": int(counts["Neutral "]),
        "n%": float(perc["Neutral "]),

        "counter": int(counts["Counter "]),
        "c%": float(perc["Counter "]),
    })

label_distribution_table = (
    pd.DataFrame(rows)
    .sort_values("total_annotations", ascending=False)
    .reset_index(drop=True)
)

label_distribution_table


,religion,total_annotations,hate,h%,neutral,n%,counter,c%
0,christian,58573,21111,36.04,14368,24.53,23094,39.43
1,nothing,36790,13285,36.11,8814,23.96,14691,39.93
2,atheist,27143,9984,36.78,6453,23.77,10706,39.44
3,other,8511,3098,36.40,2073,24.36,3340,39.24


In [27]:
df["hate_speech_score"] = pd.to_numeric(df["hate_speech_score"], errors="coerce")
df["is_hate_speech"] = (df["hate_speech_score"] > 0.5).astype(int)

df["is_hate_speech"].value_counts()

is_hate_speech
0    86508
1    49048
Name: count, dtype: int64

In [ ]:
# just for understanding
# α value	Interpretation
# < 0.20	Poor agreement
# 0.20–0.40	Fair
# 0.40–0.60	Moderate
# 0.60–0.80	Substantial
# > 0.80	Strong


In [33]:
religion_cols = [
    'annotator_religion_atheist',
    'annotator_religion_christian',
    'annotator_religion_nothing',
    'annotator_religion_other'
]
rows = []

for rel in religion_cols:
    rel_name = rel.replace("annotator_religion_", "")
    
    subset = df[df[rel] == 1]

    grouped = subset.groupby("comment_id")["is_hate_speech"].apply(list)
    grouped = grouped[grouped.apply(len) >= 2]

    if len(grouped) < 10:
        rows.append({
            "religion": rel_name,
            "comments": len(grouped),
            "krippendorff_alpha": np.nan
        })
        continue

    max_len = grouped.apply(len).max()
    reliability_matrix = [
        labels + [np.nan] * (max_len - len(labels))
        for labels in grouped
    ]

    alpha = krippendorff.alpha(
        reliability_data=reliability_matrix,
        level_of_measurement="nominal"
    )

    rows.append({
        "religion": rel_name,
        "comments": len(grouped),
        "krippendorff_alpha": round(alpha, 3)
    })

krippendorff_table = (
    pd.DataFrame(rows)
    .sort_values("comments", ascending=False)
    .reset_index(drop=True)
)

krippendorff_table


,religion,comments,krippendorff_alpha
0,christian,10433,0.081
1,nothing,4813,0.093
2,atheist,2974,0.084
3,other,348,0.024
